In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import os
import zipfile
local_zip = '/content/drive/My Drive/Train Images.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/Train Images')
local_zip ='/content/drive/My Drive/Validation Images.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/content/Validation Images')
local_zip ='/content/drive/My Drive/Test Images.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/content/Test Images')


zip_ref.close()



    

In [0]:
train_large_dir = os.path.join('/content/Train Images/Train Images/Large')

train_small_dir= os.path.join('/content/Train Images/Train Images/Small')

print('total training Large images:', len(os.listdir(train_large_dir)))
print('total training Small images:', len(os.listdir(train_small_dir)))

validation_large_dir = os.path.join('/content/Validation Images/Validation Images/Large')

validation_small_dir = os.path.join('/content/Validation Images/Validation Images/Small')

print('total validating Large images:',len(os.listdir(validation_large_dir)))
print('total validating Small images:',len(os.listdir(validation_small_dir)))


    

total training Large images: 5000
total training Small images: 5000
total validating Large images: 999
total validating Small images: 999


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,)

train_generator = train_datagen.flow_from_directory(
        '/content/Train Images/Train Images',  
        target_size=(300, 300),  
        batch_size=128,
      
        class_mode='binary')

validation_datagen = ImageDataGenerator(rescale =1./255)
validation_generator = validation_datagen.flow_from_directory('/content/Validation Images/Validation Images',target_size=(300,300),batch_size=28,class_mode= 'binary')


Found 10000 images belonging to 2 classes.
Found 1998 images belonging to 2 classes.


In [0]:
model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
  
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
  
  
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 147, 147, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 78400)            

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer='AdaGrad',
              metrics=['acc'])

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=5,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps =10)


Epoch 1/15
5/5 [==============================] - 31s 6s/step - loss: 1.5720 - acc: 0.5344 - val_loss: 0.6043 - val_acc: 0.7000
Epoch 2/15
5/5 [==============================] - 27s 5s/step - loss: 0.5837 - acc: 0.7000 - val_loss: 0.5435 - val_acc: 0.7321
Epoch 3/15
5/5 [==============================] - 27s 5s/step - loss: 0.5399 - acc: 0.7328 - val_loss: 0.5048 - val_acc: 0.7679
Epoch 4/15
5/5 [==============================] - 27s 5s/step - loss: 0.5375 - acc: 0.7406 - val_loss: 0.4845 - val_acc: 0.7571
Epoch 5/15
5/5 [==============================] - 26s 5s/step - loss: 0.4818 - acc: 0.7703 - val_loss: 0.4833 - val_acc: 0.7643
Epoch 6/15
5/5 [==============================] - 26s 5s/step - loss: 0.4517 - acc: 0.8016 - val_loss: 0.4237 - val_acc: 0.8071
Epoch 7/15
5/5 [==============================] - 27s 5s/step - loss: 0.4315 - acc: 0.8281 - val_loss: 0.4596 - val_acc: 0.7714
Epoch 8/15
5/5 [==============================] - 26s 5s/step - loss: 0.4430 - acc: 0.7844 - val_loss: 0

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import pandas as pd

filenames = pd.read_csv('/content/test.csv')

df = pd.DataFrame(columns=['Image_File', 'Class'])
for file in filenames['Image_File']:

    fn=file

  
    path = '/content/Test Images/Test Images/'+fn
    img = image.load_img(path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=50)
  
    if classes[0]>0.5:
      print(fn + ",Small")
      df = df.append({'Image_File': fn,'Class': 'Small'}, ignore_index=True)
      
    else:
      print(fn + ",Large")
      df = df.append({'Image_File': fn,'Class': 'Large'}, ignore_index=True)
      


lg 988 (1).png,Small
lg 988 (10).png,Small
lg 988 (100).png,Small
lg 988 (101).png,Small
lg 988 (102).png,Small
lg 988 (103).png,Small
lg 988 (104).png,Small
lg 988 (105).png,Large
lg 988 (106).png,Small
lg 988 (107).png,Small
lg 988 (108).png,Small
lg 988 (109).png,Small
lg 988 (11).png,Small
lg 988 (110).png,Small
lg 988 (111).png,Small
lg 988 (112).png,Small
lg 988 (113).png,Small
lg 988 (114).png,Small
lg 988 (115).png,Small
lg 988 (116).png,Small
lg 988 (117).png,Small
lg 988 (118).png,Small
lg 988 (119).png,Small
lg 988 (12).png,Small
lg 988 (120).png,Large
lg 988 (121).png,Small
lg 988 (122).png,Small
lg 988 (123).png,Small
lg 988 (124).png,Small
lg 988 (125).png,Small
lg 988 (126).png,Small
lg 988 (127).png,Small
lg 988 (128).png,Small
lg 988 (129).png,Small
lg 988 (13).png,Small
lg 988 (130).png,Large
lg 988 (131).png,Large
lg 988 (132).png,Small
lg 988 (133).png,Small
lg 988 (134).png,Small
lg 988 (135).png,Small
lg 988 (136).png,Small
lg 988 (137).png,Small
lg 988 (138).png,

In [0]:
df.to_csv('Output.csv')